In [1]:
#Import all the tools we'll need
import pandas as pd
import shapely as sp
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
from itertools import combinations
import intake
from shapely.geometry import MultiPoint
import matplotlib.pyplot as plt
import functools

C:\Users\Zade Alfalah\AppData\Local\conda\conda\envs\py37\lib\site-packages\dask\config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
#because of update to p4tools, 6/27/19
from p4tools import io

In [3]:
#because of update to p4tools
tiles = io.get_tile_coordinates(update=True)

In [4]:
#ensure tiles loaded in correctly
tiles.head()

,x_tile,y_tile,x_hirise,y_hirise,PlanetocentricLatitude,PlanetographicLatitude,PositiveEast360Longitude,BodyFixedCoordinateX,BodyFixedCoordinateY,BodyFixedCoordinateZ,tile_id,obsid
0,1,1,420.0,324.0,-85.504144,-85.556701,104.952104,-68.380674,256.056147,-3370.636698,APF0000cwz,ESP_012079_0945
1,1,2,420.0,872.0,-85.502300,-85.554878,104.934805,-68.331337,256.181661,-3370.628630,APF0000ck9,ESP_012079_0945
2,1,3,420.0,1420.0,-85.500454,-85.553054,104.917486,-68.281853,256.307249,-3370.620295,APF0000cty,ESP_012079_0945
3,1,4,420.0,1968.0,-85.498607,-85.551229,104.900198,-68.232522,256.433112,-3370.615690,APF0000ciy,ESP_012079_0945
4,1,5,420.0,2516.0,-85.496762,-85.549404,104.882968,-68.183281,256.558509,-3370.606455,APF0000cwp,ESP_012079_0945


In [5]:
#check the tile_coordinates csv, see no X,Y,Z bodyfixed coords.
tiles[tiles['obsid'] == 'ESP_012884_0935' ].head()

,x_tile,y_tile,x_hirise,y_hirise,PlanetocentricLatitude,PlanetographicLatitude,PositiveEast360Longitude,BodyFixedCoordinateX,BodyFixedCoordinateY,BodyFixedCoordinateZ,tile_id,obsid
4777,1,1,420.0,324.0,-86.455091,-86.496595,NaN,NaN,NaN,NaN,APF0000dsd,ESP_012884_0935
4778,1,2,420.0,872.0,-86.452087,-86.493626,NaN,NaN,NaN,NaN,APF0000dqa,ESP_012884_0935
4779,1,3,420.0,1420.0,-86.449083,-86.490657,NaN,NaN,NaN,NaN,APF0000dru,ESP_012884_0935
4780,1,4,420.0,1968.0,-86.446077,-86.487686,NaN,NaN,NaN,NaN,APF0000dpx,ESP_012884_0935
4781,1,5,420.0,2516.0,-86.443066,-86.484709,NaN,NaN,NaN,NaN,APF0000dsb,ESP_012884_0935


In [6]:
#updating n_obj_df (from calculate_fractional_coverage_new notebook)
olddatafull = pd.read_csv("n_obj_df.csv") #old dataframe we're comparing to
olddata=olddatafull[olddatafull['MY'] == 29] #we only want to look at MY29

In [7]:
#pull out neames of all regions 
allregions = olddata['region_name'].unique()

In [8]:
#Michael's code, gets the polygon of an obsid
def get_outer_polygon_for_obsid(obsid, return_hull=True):
    tiles = pd.read_csv("tile_coordinates.csv")
    obsid_tiles = tiles.query("obsid == @obsid")
    cols = "BodyFixedCoordinateX BodyFixedCoordinateY".split()
    coords = obsid_tiles[cols]
    coords.columns = ["x", "y"]
    mp = MultiPoint(list(zip(coords.x, coords.y)))
    if return_hull:
        return mp.convex_hull
    else:
        return mp

In [9]:
#Just gives intersection area between two polygons, docstring from .intersection just def function for .reduce later.
def oneinter(pA, pB):
    oneint = pA.intersection(pB)
    return oneint

In [10]:
#Now with reduce, self made function
#TODO: Write docstring
def reducechecker(polygon_array):
    """
    
    """
    #This block below gets our combinations of polygons, works as intended
    bucket2 = []
    combos = []
    iterable1 = range(len(polygon_array)) 
    p = len(polygon_array)
    for n in iterable1:
        if p > 0:
            comb = list(combinations(iterable1,p))
            for item in comb:
                x = item
                combos.append(x)
            p = p-1
        else:
            end
    
    
    for ele in combos:
        #create a blank dictionary
        d = {}
        #create an empty bucket
        bucket = []
        for number in ele:
            #where number would be (for example) polygon0
            bucket.append(polygon_array[number])
        #bucket now [poly0, poly1, poly2, poly4] if using comb[0] above
        #give the dictionary the polygons used from bucket
        d['elements_used'] = bucket
        #create a multipolygon out of the polygons used
        Multi = MultiPolygon(bucket)
        #create a dictionary input of the overall multipolygon
        d['multipolygon'] = Multi
        #if theres only one polygon in our bucket, make the area that polygons area
        if len(bucket) == 1:
             d['combo_area'] = bucket[0].area
        #if theres more than one polygon in the bucket, just .reduce on individual polygon intersections within the bucket
        else:
            r = functools.reduce(oneinter, bucket)
            #call combo area the final (.reduce'd) polygon's intersection
            d['combo_area'] = r.area
        #Put our bucket together
        bucket2.append(d)
    #create dataframe from bucket2
    reducedf = pd.DataFrame(bucket2)
    #spit out that dataframe
    return reducedf

Now know which regions are broken: Manhattan Classic, Giza, Caterpillar, Manhhattan Cracks, Schenec, Cortland.  All give the same error: 'Point' object does not support indexing
Some of them also have multiple, large hirise images within the region, unsure why its broken.  Don't see any points within the region at all.  
Caterpillar has just one hirise image, might be useful for debugging

In [11]:
#Some regions dont work, had to go through one by one
#TODO: Figure out exactly which regions don't work, figure out why, fix

for region in allregions[13:14]:
    #print region name for testing
    print(region)
    #create empty bucket
    bucket = []
    #go through every unique obsid
    for obs in olddata[olddata['region_name'] == region].obsid.unique():
        d2 = {}
        d2['region_name'] = region
        d2['Obsid_Used'] = obs
        x = get_outer_polygon_for_obsid(obs)
        area = x.area
        d2['Obsid_Area'] = area
        d2['polygon_shape'] = x
        bucket.append(d2)
    df1 = pd.DataFrame(bucket)
    df3 = reducechecker(df1.polygon_shape)
    bucket2 = []
    for ele in df3.index:
        #print(ele)
        polylist = df3.elements_used[ele]
        bucket = []
        for poly in polylist:
            x = df1.Obsid_Used[df1['polygon_shape'] == poly].values[0]
            bucket.append(x)
        bucket2.append(bucket)
    df3['obsids_used'] = np.array(bucket2)
    for i in range(len(df3.elements_used)): 
        df3.at[i, 'number_hirise'] = int(len(df3.elements_used[i]))
    bucket2 = []
    for obslist in df3.obsids_used:
        bucket = []
        for obs in obslist:
            x = olddata.Ls[olddata['obsid'] == obs].values[0]
            bucket.append(x)
        bucket = tuple(bucket)
        bucket2.append(bucket)
    df3['Ls_used'] = np.array(bucket2)
    df3['region_name'] = region
    df3 = df3.drop(['elements_used'], axis = 1) 
    df3.to_csv(f"{region}_combos.csv", index=False)

Giza


TypeError: 'Point' object does not support indexing

In [12]:
len(allregions)

25

In [13]:
allregions[12:13]

array(['Inca_City'], dtype=object)

In [14]:
allregions[12]

'Inca_City'

In [15]:
checkdf_inc = pd.read_csv("Inca_City_combos.csv")

In [16]:
nzcheck = checkdf_inc[checkdf_inc['combo_area'] != 0]

In [17]:
nzcheck

,combo_area,multipolygon,obsids_used,number_hirise,Ls_used,region_name
6,0.537505,"MULTIPOLYGON (((221.3919022 -461.842414, 220.2...","['ESP_012256_0985', 'ESP_012744_0985', 'ESP_01...",7.0,"(223.31400000000002, 247.27, 233.613, 265.496,...",Inca_City
19,0.557676,"MULTIPOLYGON (((221.3919022 -461.842414, 220.2...","['ESP_012256_0985', 'ESP_012744_0985', 'ESP_01...",6.0,"(223.31400000000002, 247.27, 233.613, 265.496,...",Inca_City
20,0.556264,"MULTIPOLYGON (((221.3919022 -461.842414, 220.2...","['ESP_012256_0985', 'ESP_012744_0985', 'ESP_01...",6.0,"(223.31400000000002, 247.27, 233.613, 265.496,...",Inca_City
21,0.668095,"MULTIPOLYGON (((221.3919022 -461.842414, 220.2...","['ESP_012256_0985', 'ESP_012744_0985', 'ESP_01...",6.0,"(223.31400000000002, 247.27, 233.613, 265.496,...",Inca_City
22,1.088736,"MULTIPOLYGON (((221.3919022 -461.842414, 220.2...","['ESP_012256_0985', 'ESP_012744_0985', 'ESP_01...",6.0,"(223.31400000000002, 247.27, 233.613, 198.3179...",Inca_City
23,0.565751,"MULTIPOLYGON (((221.3919022 -461.842414, 220.2...","['ESP_012256_0985', 'ESP_012744_0985', 'ESP_01...",6.0,"(223.31400000000002, 247.27, 265.496, 198.3179...",Inca_City
29,0.537505,"MULTIPOLYGON (((221.3919022 -461.842414, 220.2...","['ESP_012256_0985', 'ESP_012467_0985', 'ESP_01...",6.0,"(223.31400000000002, 233.613, 265.496, 198.317...",Inca_City
35,0.537505,"MULTIPOLYGON (((221.0729768 -461.0059403, 221....","['ESP_012744_0985', 'ESP_012467_0985', 'ESP_01...",6.0,"(247.27, 233.613, 265.496, 198.31799999999998,...",Inca_City
47,0.891139,"MULTIPOLYGON (((221.3919022 -461.842414, 220.2...","['ESP_012256_0985', 'ESP_012744_0985', 'ESP_01...",5.0,"(223.31400000000002, 247.27, 233.613, 265.496,...",Inca_City
48,0.783563,"MULTIPOLYGON (((221.3919022 -461.842414, 220.2...","['ESP_012256_0985', 'ESP_012744_0985', 'ESP_01...",5.0,"(223.31400000000002, 247.27, 233.613, 265.496,...",Inca_City


In [18]:
nzcheck.multipolygon[6]

'MULTIPOLYGON (((221.3919022 -461.842414, 220.2338969 -449.8576356, 220.4156555 -449.8358091, 220.9617314 -449.7776505, 221.3777812 -454.0387772, 221.4293569 -454.570284, 221.4422228 -454.7029279, 221.6872346 -457.2408013, 221.712342 -457.5019148, 221.7757466 -458.1616486, 221.8912803 -459.3711563, 222.1072974 -461.6356416, 222.1198629 -461.7675375, 221.3919022 -461.842414)), ((221.0729768 -461.0059403, 221.0501253 -460.4620796, 220.9088843 -457.0605937, 220.7434818 -452.8237469, 220.7139373 -451.9995734, 220.6917217 -451.310346, 221.4387743 -451.2865015, 221.5134014 -452.9126517, 221.5741768 -454.2697998, 221.72577 -458.0929097, 221.8116798 -460.5624402, 221.8258459 -460.9746643, 221.2595547 -461.000212, 221.0729768 -461.0059403)), ((221.0314688 -458.878432, 220.9380838 -457.8188208000001, 220.926488 -457.6871428, 220.9149285 -457.5552793000001, 220.8006328 -456.2215762, 220.7891837 -456.0877524, 220.6279373 -454.201603, 220.9930205 -454.1654777, 221.1755411 -454.1487635, 221.3579349 